# Naïve Bais For Spam and Non Spam Classification

In [1]:
import glob
import numpy as np
import pandas as pd
import os
import random

In [2]:
data_directory = 'Task6_DataSet'
os.chdir(data_directory)

In [3]:
print(os.getcwd())

/Users/shivam/Desktop/Learning/Semester II/Machine Learning/Lab/Task 1/Task6_DataSet


In [4]:
# Generate a random number to use it as a testing dataset
testing_set_number = random.randint(1,10)
print('We Will use Part{} as test dataset'.format(testing_set_number))

We Will use Part9 as test dataset


In [5]:
def populate_data(data_dictionary, filename):
    with open(filename) as file:
        file_data = []
        for line in file:
            if line:
                if 'Subject' in line:
                    data_dictionary['Subject'].append(line.split(':')[1].strip())
                else:
                    file_data.append(line.strip())

        data_dictionary['message'].append(''.join(file_data))
        if 'legit' in filename:
            data_dictionary['spam'].append(0)
        else:
            data_dictionary['spam'].append(1)
        data_dictionary['filename'].append(filename)
        
        

In [6]:
# Generate out DataFrame
X = {'filename': [], 'Subject' : [], 'message' : [], 'spam' : []}
Y = {'filename': [], 'Subject' : [], 'message' : [], 'spam' : []}


for file in glob.iglob('**/*', recursive=True):
    if os.path.isfile(file):
        if 'part{}/'.format(testing_set_number) in file:
            populate_data(Y, file)
        else:
            populate_data(X, file)
        
            
        

In [9]:
X_data = pd.DataFrame(X)
Y_data = pd.DataFrame(Y)
print(X_data.shape, Y_data.shape)


(981, 4) (109, 4)


In [10]:
X_data.head(10)

,filename,Subject,message,spam
0,part3/31038legit19.txt,2056 118 20153 11852,284 6818 80 284 13383 80 127 93 84 489 18798 2...,0
1,part3/31024legit30.txt,16351 9568 118 3842 17730 7634 14338 103 69,16502 21946 7634 16893 15149 1613 16538 80 115...,0
2,part3/3435legit3.txt,167 14444 118 1686 118 19310 82 15527 18916 90...,14338 7488 2221 20439 103 80 12116 18469 18706...,0
3,part3/3707spmsg74.txt,1968 21501 80 1968 20235 47 47 47,4822 80 16502 2410 1967 180 262 1847 12146 199...,1
4,part3/31017spmsg80.txt,130 8857,478 17188 130 20259 12808 80 196 17054 1812 17...,1
5,part3/3762legit23.txt,8197 15018,16147 17345 14338 14934 17544 15511 437 80 176...,0
6,part3/3480spmsg75.txt,6306 118 86 86 17212 84 11835 84 1694,16502 11317 160 80 17336 23422 18104 22913 199...,1
7,part3/3987legit52.txt,2056 118 13547 5228 1613 10898 51 1465 51,284 14851 80 196 4339 6427 17753 17345 180 725...,0
8,part3/3285legit50.txt,167 190 82 14988 171 20428 1835 2418 82 7967 3...,20428 1835 2418 82 7967 3788 8584 16502 21856 ...,0
9,part3/3201spmsg99.txt,15527 1613 20382 84 13830,19054 1847 15527 1613 6306 118 86 86 20382 84 ...,1


In [11]:
 Y_data.head(10)

,filename,Subject,message,spam
0,part9/932spmsg103.txt,19054 1847 15527 1613 20382 84 13830,1692 14342 21366 2130 12424 16887 21013 131 76...,1
1,part9/9195spmsg61.txt,775 13185 13476 180 13747 47 47,23607 12497 18957 19054 1847 180 15820 19206 1...,1
2,part9/919spmsg106.txt,1692 17345 10936 11279 4774 131,1791 80 5657 17345 9472 2130 1192 1593 10299 5...,1
3,part9/9149spmsg72.txt,178,19071 47 526 16250 15964 16615 2521 24640 9494...,1
4,part9/9517legit3.txt,5652 13547 5228 1613 10898,284 1705 84 183 84 187 84 1494 118 20684 196 1...,0
5,part9/9848legit6.txt,2056 68 966 70 118 20153 11852,284 1294 80 196 1591 8962 22641 4177 180 20215...,0
6,part9/9824spmsg69.txt,2176 64 2056 19884 80 10591 13406 5118 18948 3...,2176 64 2056 19884 80 10591 13406 5118 18948 3...,1
7,part9/9450spmsg78.txt,2056 118 11521 13785 2956,15800 22232 22752 219 187 183 196 93 86 101 68...,1
8,part9/9835legit58.txt,167 8 171 2750 64 964 13651 82 3696 62 20644 1...,13207 64 964 23681 14338 1359 15671 4318 20215...,0
9,part9/9277legit20.txt,180 22872 20908 16502 16032,284 13383 80 15820 2008 16811 17157 68 9599 16...,0


In [12]:
from collections import defaultdict

def count_it(column):
    counter = defaultdict(int)
    for row in column:
        words = row.split()
        for word in words:
            counter[word] += 1
    return dict(counter)



def get_spam_and_legit_dictionary(data_frame):
    spam_data = data_frame.loc[data_frame['spam'] == 1]
    legit_data = data_frame.loc[data_frame['spam'] == 0]
    spam_subject_counter = count_it(spam_data['Subject'])
    spam_message_counter = count_it(spam_data['message'])
    legit_subject_counter = count_it(legit_data['Subject'])
    legit_message_counter = count_it(legit_data['message'])
    return spam_subject_counter, spam_message_counter, legit_subject_counter, legit_message_counter


In [13]:
spam_subject_count, spam_message_count, legit_subject_count, legit_message_count = get_spam_and_legit_dictionary(X_data)

In [53]:
def convert_values_to_probability(count_dictionary):
    total_number = sum(count_dictionary.values())
    return {k:v/total_number for k,v in count_dictionary.items()}

In [41]:
spam_subject = convert_values_to_probability(spam_subject_count)
spam_message = convert_values_to_probability(spam_message_count)
legit_subject = convert_values_to_probability(legit_subject_count)
legit_message = convert_values_to_probability(legit_message_count)

In [54]:
def probability_of_occurance(data_row, probability_dictionary):
    probability = 1
    words = data_row.split()
    for word in words:
        if word in probability_dictionary:
            probability += probability_dictionary[word]
            
    return probability
                

In [71]:
Y_predict = []
for i in range(len(Y_data)):
    data_row = Y_data.iloc[i]
    subject_probability_spam = probability_of_occurance(data_row['Subject'], spam_subject)
    subject_probability_legit = probability_of_occurance(data_row['Subject'], legit_subject)
    message_probability_spam = probability_of_occurance(data_row['message'], spam_message)
    message_probability_legit = probability_of_occurance(data_row['message'], legit_message)
#     print(subject_probability_spam, subject_probability_legit, message_probability_legit, message_probability_spam)
    if (2*subject_probability_spam + message_probability_spam) >= (2*subject_probability_legit + message_probability_legit):
        Y_predict.append(1)
    else:
        Y_predict.append(0)
Y_predict = np.array(Y_predict)

In [72]:
Y_predict = np.array(Y_predict)

In [73]:
print(Y_data['spam'].iloc[0:3])
print(Y_predict[0:3])

0    1
1    1
2    1
Name: spam, dtype: int64
[1 1 1]


In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_data['spam'], Y_predict)*100

79.81651376146789